In [1]:
import MySQLdb
import csv
import os
import subprocess
import pandas as pd

In [2]:
def write_header():
    head = ["cid","time","compile","vmlinux","GZIP-bzImage","GZIP-vmlinux","GZIP","BZIP2-bzImage","BZIP2-vmlinux","BZIP2","LZMA-bzImage","LZMA-vmlinux","LZMA","XZ-bzImage","XZ-vmlinux","XZ","LZO-bzImage","LZO-vmlinux","LZO","LZ4-bzImage","LZ4-vmlinux","LZ4","basic-vmlinux","basic-GZIP-bzImage","basic-GZIP-vmlinux","basic-GZIP","basic-BZIP2-bzImage","basic-BZIP2-vmlinux","basic-BZIP2","basic-LZMA-bzImage","basic-LZMA-vmlinux","basic-LZMA","basic-XZ-bzImage","basic-XZ-vmlinux","basic-XZ","basic-LZO-bzImage","basic-LZO-vmlinux","basic-LZO","basic-LZ4-bzImage","basic-LZ4-vmlinux","basic-LZ4"]
    with open("stats.csv", 'w') as file:
        writer = csv.DictWriter(file,head)
        writer.writeheader()

In [3]:
def kernel2csv(compressed):
    compressed = compressed.split(" , ")
    parse_array = [i.split(" : ") for i in compressed]
    final = [fin[1] for fin in parse_array]

    return final

In [4]:
def compute_rows():
    try:
        socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT cid, compilation_time, core_size, compressed_sizes FROM Compilations WHERE cid >= 20000 ORDER BY cid DESC"
        cursor.execute(query)
        entry = cursor.fetchall()
    except:
        print("Error")
    finally:
        cursor.close()
        socket.close()
    
    return entry

In [5]:
## Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [6]:
def create_csv():
    
    subprocess.run("rm stats.csv", shell=True)
    if not os.path.exists("stats.csv"):
        write_header()  
        
    entry_unformatted = compute_rows()
    entry = []
    for i in range(len(entry_unformatted)):
        compr = kernel2csv(entry_unformatted[i][3])
        entry.append(list(entry_unformatted[i][0: 3]) + compr)
    
    with open("stats.csv", 'a') as file:
        writer = csv.writer(file)
        for line in entry:
            writer.writerow(line)
    subprocess.run('wc -c stats.csv', shell=True)
    

In [7]:
def main():  
    create_csv()
    min_max()
    

In [8]:
def min_max():
    df = pd.read_csv('stats.csv')
    size_max = max(df["vmlinux"])
    size_min = min(df["vmlinux"].where(df["vmlinux"] >= 0))
    tmp_size_average = df["vmlinux"].where(df["vmlinux"] >= 0).mean()
    size_average = float(str(tmp_size_average).split('.')[0] + '.' + str(tmp_size_average).split('.')[1][:3])
    
    time_max = max(df["time"])
    time_min = min(df["time"].where(df["time"] >= 0))
    tmp_time_average = df["time"].where(df["time"] >= 0).mean()
    time_average = float(str(tmp_time_average).split('.')[0] + '.' + str(tmp_time_average).split('.')[1][:3])
    
    print("Number of values:", len(df))
    
    print("Time Max:", time_max, end="")
    print(" Time Min:", time_min, end="")
    print(" Time average:", time_average)
    
    print("Size Max:", size_max, end="")
    print(" Size Min:", size_min, end="")
    print(" Size average:", size_average)
        

In [9]:
main()

Number of values: 36583
Time Max: 4278.96 Time Min: 86.3592 Time average: 399.302
Size Max: 108607024.0 Size Min: 2126800.0 Size average: 10594460.713
